In [1]:
!pip install mediapipe opencv-python numpy

  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.6.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.6.1-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached matplotlib-3.10.3-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.7-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import socket

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

serverAddressPort = ('127.0.0.1', 12345)
sock.connect(serverAddressPort)

with mp_hands.Hands() as hands:

    while cap.isOpened():

        ret, frame = cap.read()

        if not ret:
            print("ignoring empty frame")
            continue

        image = frame
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand)
                print(hand)
                sock.send(str.encode(str(results.multi_hand_world_landmarks)))

        cv2.imshow("hand tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            sock.close()
            break
cap.release()
cv2.destroyAllWindows()

landmark {
  x: 0.955939472
  y: 0.33621484
  z: 1.40214161e-006
}
landmark {
  x: 0.857669711
  y: 0.198881477
  z: -0.137396604
}
landmark {
  x: 0.761564
  y: 0.21720022
  z: -0.244801238
}
landmark {
  x: 0.671889246
  y: 0.298950851
  z: -0.318228215
}
landmark {
  x: 0.606106043
  y: 0.398691714
  z: -0.396685928
}
landmark {
  x: 0.684632361
  y: 0.314339817
  z: -0.363067806
}
landmark {
  x: 0.53460896
  y: 0.391012579
  z: -0.501781225
}
landmark {
  x: 0.426284254
  y: 0.463957816
  z: -0.585500419
}
landmark {
  x: 0.328829795
  y: 0.534704804
  z: -0.638708115
}
landmark {
  x: 0.687532544
  y: 0.471671104
  z: -0.329905182
}
landmark {
  x: 0.500226736
  y: 0.603212178
  z: -0.475248396
}
landmark {
  x: 0.365775436
  y: 0.723175764
  z: -0.556002319
}
landmark {
  x: 0.251805216
  y: 0.830154896
  z: -0.611985862
}
landmark {
  x: 0.716420889
  y: 0.605530441
  z: -0.288126916
}
landmark {
  x: 0.548154414
  y: 0.730748296
  z: -0.408067554
}
landmark {
  x: 0.420023024
